In [1]:
!wget https://zenodo.org/records/3723295/files/annotations.csv

--2024-11-03 09:49:17--  https://zenodo.org/records/3723295/files/annotations.csv
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136986 (134K) [text/plain]
Saving to: 'annotations.csv'

annotations.csv     100%[===================>] 133.78K   207KB/s    in 0.6s    

2024-11-03 09:49:19 (207 KB/s) - 'annotations.csv' saved [136986/136986]



In [2]:
!wget https://zenodo.org/records/3723295/files/subset5.zip

--2024-11-03 09:49:20--  https://zenodo.org/records/3723295/files/subset5.zip
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6610460097 (6.2G) [application/octet-stream]
Saving to: 'subset5.zip'

subset5.zip         100%[===================>]   6.16G  13.7MB/s    in 7m 21s  

2024-11-03 09:56:42 (14.3 MB/s) - 'subset5.zip' saved [6610460097/6610460097]



In [3]:
!unzip -q /kaggle/working/subset5.zip -d /kaggle/working/

In [4]:
!rm subset5.zip

In [5]:
import SimpleITK as sitk
import numpy as np
import csv
from glob import glob
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

In [6]:
# LUNA_mask_extraction.py
subset = "subset5"
file_list = os.listdir(f"/kaggle/working//{subset}")
file_list = list(map(lambda file : f"/kaggle/working/{subset}/" + file, file_list))

In [7]:
def myFunc(file):
  if file.endswith(".mhd"):
    return True
  else:
    return False

file_list = list(filter(myFunc, file_list))

In [8]:
!mkdir ct_images

In [9]:
def make_mask(center, diam, z, width, height, spacing, origin):
    '''
        Center : centers of circles px -- list of coordinates x,y,z
        diam : diameters of circles px -- diameter
        widthXheight : pixel dim of image
        spacing = mm/px conversion rate np array x,y,z
        origin = x,y,z mm np.array
        z = z position of slice in world coordinates mm
    '''
    mask = np.zeros([height, width]) # 0's everywhere except nodule swapping x,y to match img
    #convert to nodule space from world coordinates

    # Defining the voxel range in which the nodule falls
    v_center = (center - origin)/spacing
    v_diam = int(diam/spacing[0] + 1)
    v_xmin = np.max([0, int(v_center[0]-v_diam/2) - 2])
    v_xmax = np.min([width-1, int(v_center[0]+v_diam/2) + 2])
    v_ymin = np.max([0, int(v_center[1]-v_diam/2) - 2]) 
    v_ymax = np.min([height-1, int(v_center[1]+v_diam/2) + 2])

    v_xrange = range(v_xmin, v_xmax+1)
    v_yrange = range(v_ymin, v_ymax+1)

    # Fill in 1 within sphere around nodule
    for v_x in v_xrange:
        for v_y in v_yrange:
            p_x = spacing[0]*v_x + origin[0]
            p_y = spacing[1]*v_y + origin[1]
            if np.linalg.norm(center - np.array([p_x, p_y, z])) <= diam:
                mask[int((p_y-origin[1]) / spacing[1]), int((p_x-origin[0]) / spacing[0])] = 1.0
    return(mask, [v_xmin, v_xmax, v_ymin, v_ymax])

def get_filename(case):
    global file_list
    for f in file_list:
        if case in f:
            return(f)
        
def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

In [10]:
if __name__ == "__main__":
    df_node = pd.read_csv("/kaggle/working/annotations.csv")
    df_node["file"] = df_node["seriesuid"].apply(get_filename)
    df_node = df_node.dropna()
    # Define the file name
    file_name = 'output.csv'

    # Define the column names
    columns = ['x_min', 'x_max', 'y_min', 'y_max', 'ID']

    # Create an empty CSV file with column headers
    with open(file_name, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=columns)
        writer.writeheader()
    df_roi = pd.read_csv("output.csv")
    df_roi_cur = dict()
    
    print(f"TRÍCH XUẤT ẢNH VÀ TỌA ĐỘ TỪ FILE .raw VÀ FILE .mhd TỪ {subset}")
    for fcount, img_file in enumerate(tqdm(file_list)):
        mini_df = df_node[df_node["file"]==img_file] # get all nodules associate with file
        if len(mini_df) > 0:       # some files may not have a nodule--skipping those
            itk_img = sitk.ReadImage(img_file) # read .mhd file
            img_array = sitk.GetArrayFromImage(itk_img) # indexes are z,y,x (notice the ordering)
            num_z, height, width = img_array.shape
            origin = np.array(itk_img.GetOrigin())      # x,y,z  Origin in world coordinates (mm)
            spacing = np.array(itk_img.GetSpacing())    # spacing of voxels in world coor. (mm)
            
            for node_idx, cur_row in mini_df.iterrows():       
                node_x = cur_row["coordX"]
                node_y = cur_row["coordY"]
                node_z = cur_row["coordZ"]
                diam = cur_row["diameter_mm"]
                
                # imgs = np.ndarray([num_z, height, width], dtype=np.float16)
                center = np.array([node_x, node_y, node_z])   # nodule center
                v_center = np.rint((center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            
                for i_z in range(0, num_z): # clip prevents going out of bounds in Z
                    _, roi = make_mask(center, diam, i_z*spacing[2]+origin[2], width, height, spacing, origin)
                    if i_z == v_center[2]:
                        roi_key = f"{img_file.split('/')[-1].rsplit('.', 1)[0]}_{i_z}_{node_idx}"
                        img_name = f"{img_file.split('/')[-1].rsplit('.', 1)[0]}_{i_z}_{node_idx}.png"
                        img = normalize(img_array[i_z])
                        img = img * 255
                        img_rgb = np.stack((img,)*3, -1)
                        cv2.imwrite(f"/kaggle/working/ct_images/{img_name}", img_rgb)
                        df_roi_cur.update({roi_key: roi})
                    # imgs[i_z] = img_array[i_z]
    df_temp = pd.DataFrame.from_dict(df_roi_cur, columns=["x_min", "x_max", "y_min", "y_max"], orient='index')
    df_temp["ID"] = list(df_roi_cur.keys())
    df_temp = df_temp.reset_index(drop=True)
    df_roi = pd.concat([df_roi, df_temp])
    df_roi.to_csv("output.csv", index=False)

TRÍCH XUẤT ẢNH VÀ TỌA ĐỘ TỪ FILE .raw VÀ FILE .mhd TỪ subset5


100%|██████████| 89/89 [01:08<00:00,  1.31it/s]


In [11]:
!zip -r subset5.zip /kaggle/working/ct_images /kaggle/working/output.csv

  adding: kaggle/working/ct_images/ (stored 0%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588_127_497.png (deflated 19%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.178391668569567816549737454720_79_304.png (deflated 19%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.308655308958459380153492314021_182_844.png (deflated 19%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.323408652979949774528873200770_106_891.png (deflated 18%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.413896555982844732694353377538_63_974.png (deflated 20%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.296738183013079390785739615169_152_801.png (deflated 15%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.6001.255999614855292116767517149228_73_646.png (deflated 20%)
  adding: kaggle/working/ct_images/1.3.6.1.4.1.14519.5.2.1.6279.60